# Google Trends

In [1]:
import os
os.chdir("../../")

import inspect
import numpy as np
import pandas as pd
from scripts.python.GoogleTrends import GT

The following code was modified from [Development Data Partership](https://datapartnership.org/). See [example code(https://github.com/datapartnership/docs-google/blob/master/examples/google-health-trends-api.ipynb).

In [2]:
print(inspect.getsource(GT))

class GT:
    def __init__(self, _GOOGLE_API_KEY = GoogleAPIkey):
        self.service = build(
            serviceName=SERVICE_NAME,
            version=SERVICE_VERSION,
            discoveryServiceUrl=_DISCOVERY_SERVICE_URL,
            developerKey=_GOOGLE_API_KEY,
            cache_discovery=False)
        self.block_until = None

    def get_health_trends(self, terms, timelineResolution="month"):
        graph = self.service.getTimelinesForHealth(
            terms=terms,
            timelineResolution=timelineResolution
        )

        try:
            response = graph.execute()
            return response

        except HttpError as http_error:
            data = json.loads(http_error.content.decode('utf-8'))
            code = data['error']['code']
            reason = data['error']['errors'][0]['reason']
            if code == 403 and reason == 'dailyLimitExceeded':
                self.block_until = datetime.combine(
                    date.today() + timedelta(days=1), d

In [3]:
folder = os.getcwd() + "/data/tourism/trends/"
filepaths = [folder + path for path in os.listdir(folder)]
filepaths

['/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_palau.csv',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/readme.md',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_solomon_islands.csv',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_tonga.csv',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_vanuatu.csv',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_fiji.csv',
 '/Users/czhang/Desktop/pacific-observatory/data/tourism/trends/trends_samoa.csv']

In [4]:
palau_trend = pd.read_csv(filepaths[0]).drop("Unnamed: 0", axis=1)
palau_trend["date"] = pd.to_datetime(palau_trend["date"] )
palau_trend.head(5)

,date,palau flights-AU,palau hotel-AU,palau travel-AU,palau flights-NZ,palau hotel-NZ,palau travel-NZ,palau flights-US,palau hotel-US,palau travel-US,...,palau travel-JP,palau flights-HK,palau hotel-HK,palau travel-HK,palau flights-NC,palau hotel-NC,palau travel-NC,palau flights,palau hotel,palau travel
0,2004-01-01,0,0,0,0,0,0,0,23,0,...,0,0,0,0,0,0,0,0.0,3.155717,3.055298
1,2004-02-01,0,0,0,0,0,0,0,0,28,...,0,0,0,0,0,0,0,0.0,5.111154,2.142547
2,2004-03-01,0,63,0,0,0,0,0,0,0,...,0,0,90,0,0,0,0,0.0,6.132983,2.768303
3,2004-04-01,0,0,0,0,0,0,0,0,100,...,0,0,0,0,0,0,0,0.0,10.537711,0.000000
4,2004-05-01,0,0,0,0,0,0,22,0,0,...,0,0,0,0,0,0,0,0.0,7.344062,0.000000


In [5]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Legend
output_notebook()

p = figure(height=500, width=780,
           title="Google Search Trends (Palau)",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Number of Passengers Per Month",
           tooltips=[("country", "$name"),
                     ("search index", "@$name")])

keywords = ["palau hotel", "palau travel", "palau flights"]
p.add_layout(Legend(), 'right')
source = ColumnDataSource(palau_trend)

for keyword, color in zip(keywords, Category20[12]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('date', keyword, source=source, name=keyword,
           legend_label=keyword, color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)

Loading BokehJS ...